In [1]:
import glob
import os
import json
import subprocess
import pandas as pd

In [8]:
file_path = '/Users/heejungj/Documents/projects_local/visualsnow_source/sub-002/fmap/sub-002_dir-PA_run-06_epi.json'
with open(file_path, 'r') as file:
    json_data = json.load(file)

In [ ]:
json_data['info']['TotalReadoutTime']

{'AcquisitionMatrix': [88, 0, 0, 88],
 'AcquisitionMatrixPE': 88,
 'AcquisitionNumber': 1,
 'AcquisitionTime': '11:02:40.000000',
 'BodyPartExamined': 'BRAIN',
 'CoilString': '48HAP',
 'Columns': 88,
 'ConversionSoftware': 'dcm2niix',
 'ConversionSoftwareVersion': 'v1.0.20201102',
 'DeviceSerialNumber': '0000415723SHMR18',
 'EchoTime': 0.03,
 'EffectiveEchoSpacing': 0.000564,
 'FlipAngle': 62,
 'ImageOrientationPatientDICOM': [1, 0, 0, 0, 0.99458, -0.10393],
 'ImageType': ['ORIGINAL', 'PRIMARY', 'OTHER'],
 'ImagingFrequency': 127.762,
 'InPlanePhaseEncodingDirection': 'COL',
 'InPlanePhaseEncodingDirectionDICOM': 'COL',
 'InstitutionName': 'Lucas Center',
 'InternalPulseSequenceName': 'EPI',
 'MRAcquisitionType': '2D',
 'MagneticFieldStrength': 3,
 'Manufacturer': 'GE',
 'ManufacturersModelName': 'SIGNA Premier',
 'Modality': 'MR',
 'MultibandAccelerationFactor': 4,
 'NumberOfTemporalPositions': 10,
 'PatientPosition': 'HFS',
 'PercentPhaseFOV': 100,
 'PercentPhaseFieldOfView': 100,
 '

In [20]:
# Helper function to load and parse JSON files
def extract_phase_encoding_data(files, file_type):
    data = []
    for file_path in sorted(files):
        file_name = os.path.basename(file_path)
        try:
            with open(file_path, 'r') as file:
                json_data = json.load(file)
                run_name = file_name.split("_")[2]
                total_readout = json_data['info']['TotalReadoutTime']
                # Optionally, if multiple files are processed, you might want to warn if they differ
                # if total_readout is None:
                #     total_readout = json_data['info']['TotalReadoutTime']
                # elif total_readout != current_total:
                #     print(f"Warning: Different TotalReadoutTime encountered in {file_path}. Using first value.")

                if file_type == "fmap":
                    data.append({
                        "run": run_name,
                        "fmap_PhaseEncodingPolarityGE": json_data['info']['PhaseEncodingPolarityGE'],
                        "fmap_PhaseEncodingDirection": json_data['info']['PhaseEncodingDirection']
                    })
                elif file_type == "func":
                    task_name = file_name.split("_")[1]
                    data.append({
                        "task": task_name,
                        "run": run_name,
                        "bold_PhaseEncodingPolarityGE": json_data['info']['PhaseEncodingPolarityGE'],
                        "bold_PhaseEncodingDirection": json_data['info']['PhaseEncodingDirection']
                    })
        except KeyError as e:
            print(f"Missing key {e} in {file_path}. Skipping.")
        except Exception as e:
            print(f"Error processing {file_path}: {e}. Skipping.")
    return data, total_readout


In [16]:
def create_acqparams_per_run_with_opposite(merged_table, output_dir, sub, dim4, DIRECTION_MAPPING):
    for run in merged_table["run"].unique():
        run_data = merged_table[merged_table["run"] == run]

        # Generate main direction
        acqparams = []
        for _, row in run_data.iterrows():
            main_direction = DIRECTION_MAPPING.get(
                row.get("fmap_PhaseEncodingDirection", row.get("bold_PhaseEncodingDirection", "N/A")),
                "0 0 0 0"
            )
            acqparams.append(main_direction)

            # Automatically infer opposite phase encoding
            opposite_direction = " ".join(
                str(-int(value)) if i < 3 else value
                for i, value in enumerate(main_direction.split())
            )

        # Create the repeated rows for main and opposite directions
        repeated_acqparams = ([main_direction] * dim4) + ([opposite_direction] * dim4)

        # Write to the output file
        output_path = os.path.join(output_dir, f"{sub}_{run}_acqparams.txt")
        with open(output_path, "w") as file:
            file.write("\n".join(repeated_acqparams))
        print(f"acqparams.txt created for {run} at: {output_path}")




# baed on encoding direction metadata,
We create a mapping with 
a) 3 columns of phase encoding direction
b) 4th column of total readout time
- total read out time is based on `(number of echoes minus one) * echo spacing`
- In our .json files, the corresponding parameter for 
- number of echo is `AcquisitionMatrixPE`
- echo spacing is `EffectiveEchoSpacing`

- Further information: https://lcni.uoregon.edu/wiki/tags/fmri/#:~:text=The%20total%20readout%20time%20is,applying%20the%20fieldmap%20to%20later.

In [21]:
fmap_files

['/Users/heejungj/Documents/projects_local/visualsnow_source/sub-002/fmap/sub-002_dir-PA_run-11_epi.json',
 '/Users/heejungj/Documents/projects_local/visualsnow_source/sub-002/fmap/sub-002_dir-PA_run-10_epi.json',
 '/Users/heejungj/Documents/projects_local/visualsnow_source/sub-002/fmap/sub-002_dir-PA_run-06_epi.json',
 '/Users/heejungj/Documents/projects_local/visualsnow_source/sub-002/fmap/sub-002_dir-PA_run-07_epi.json',
 '/Users/heejungj/Documents/projects_local/visualsnow_source/sub-002/fmap/sub-002_dir-PA_run-01_epi.json',
 '/Users/heejungj/Documents/projects_local/visualsnow_source/sub-002/fmap/sub-002_dir-PA_run-05_epi.json',
 '/Users/heejungj/Documents/projects_local/visualsnow_source/sub-002/fmap/sub-002_dir-PA_run-04_epi.json',
 '/Users/heejungj/Documents/projects_local/visualsnow_source/sub-002/fmap/sub-002_dir-PA_run-02_epi.json',
 '/Users/heejungj/Documents/projects_local/visualsnow_source/sub-002/fmap/sub-002_dir-PA_run-03_epi.json',
 '/Users/heejungj/Documents/projects_

In [24]:
# Define constants

for sub in ['sub-002']: #['sub-001', 'sub-002','sub-003']:
    base_dir = f"/Users/heejungj/Documents/projects_local/visualsnow_source"
    fmap_path = f"{base_dir}/{sub}/fmap/{sub}_dir-PA_run-*_epi.json"
    func_path = f"{base_dir}/{sub}/func/{sub}_task-*_run-*_bold.json"
    output_dir = f"{base_dir}/acqparams_per_run/{sub}"

    # Ensure output directory exists
    os.makedirs(output_dir, exist_ok=True)


    # Extract data
    fmap_files = glob.glob(fmap_path)
    func_files = glob.glob(func_path)
    # Remember, this total readout is all from the pepolar sequence, 
    # NOT the functional sequence you will be applying the fieldmap to later
    fmap_data, total_readout = extract_phase_encoding_data(fmap_files, file_type="fmap")
    func_data, NULL = extract_phase_encoding_data(func_files, file_type="func")

    
    DIRECTION_MAPPING = {
        "i": f"1 0 0 {total_readout}",
        "i-": f"-1 0 0 {total_readout}",
        "j": f"0 1 0 {total_readout}",
        "j-": f"0 -1 0 {total_readout}",
        "k": f"0 0 1 {total_readout}",
        "k-": f"0 0 -1 {total_readout}",
    }
    # Convert to DataFrames
    fmap_table = pd.DataFrame(fmap_data)
    func_table = pd.DataFrame(func_data)


    # Merge DataFrames on "Run"
    merged_table = pd.merge(fmap_table, func_table, on="run", how="outer")
    merged_table['sub'] = sub
    # Assume df is your DataFrame
    desired_order = ['sub', 'run', 'task', 'bold_PhaseEncodingPolarityGE', 'bold_PhaseEncodingDirection', 'fmap_PhaseEncodingPolarityGE', 'fmap_PhaseEncodingDirection']
    merged_table = merged_table[desired_order]
    os.makedirs('phase_encoding', exist_ok=True)

    merged_table.to_csv(f'./phase_encoding/{sub}_phase_encoding.tsv', sep='\t', header=True, index=False)
    # merged_table
    # Pretty print merged_table
    print(f"\n\n=== Phase Encoding Table for {sub} ===")
    print(merged_table.to_string(index=False, justify='center'))

    # Call the function to generate acqparams
    with open(fmap_files[0], 'r') as file:
        check_dim4 = json.load(file)
    dim4 = check_dim4['info']['NumberOfTemporalPositions']
    create_acqparams_per_run_with_opposite(merged_table, output_dir, sub, dim4, DIRECTION_MAPPING)




=== Phase Encoding Table for sub-002 ===
  sub    run         task      bold_PhaseEncodingPolarityGE bold_PhaseEncodingDirection fmap_PhaseEncodingPolarityGE fmap_PhaseEncodingDirection
sub-002 run-01   task-restopen          Unflipped                        j-                       Flipped                         j             
sub-002 run-02 task-restclosed          Unflipped                        j-                       Flipped                         j             
sub-002 run-03 task-restclosed          Unflipped                        j-                       Flipped                         j             
sub-002 run-04   task-restopen          Unflipped                        j-                       Flipped                         j             
sub-002 run-05 task-restclosed          Unflipped                        j-                       Flipped                         j             
sub-002 run-06   task-restopen          Unflipped                        j-            

# BRAND NEW CODE

In [131]:
import pydicom
import os

# Path to your DICOM folder
dicom_folder = "/Users/heejungj/Downloads/dicom.dicom"
# /Users/heejungj/Downloads/dicom.dicom/1.2.840.113619.2.514.11554579.1340171.29576.1733166234.112.MR.dcm 
# Initialize a list to store extracted info
dicom_info = []

# Loop through each DICOM file
for dicom_file in sorted(os.listdir(dicom_folder)):
    file_path = os.path.join(dicom_folder, dicom_file)
    ds = pydicom.dcmread(file_path)

    # Extract relevant information
    phase_encoding_dir = getattr(ds, "InPlanePhaseEncodingDirection", "N/A")
    instance_number = getattr(ds, "InstanceNumber", "N/A")
    total_readout_time = getattr(ds, "TotalReadoutTime", "N/A")

    dicom_info.append({
        "File": dicom_file,
        "InstanceNumber": instance_number,
        "PhaseEncodingDirection": phase_encoding_dir,
        "TotalReadoutTime": total_readout_time
    })

# Print results
for info in dicom_info:
    print(info)

ds = pydicom.dcmread(os.path.join(dicom_folder, dicom_file))

print(f"InstanceNumber: {ds.get('InstanceNumber', 'N/A')}")
print(f"NumberOfSlices: {ds.get('NumberOfSlices', 'N/A')}")
print(f"NumberOfTemporalPositions: {ds.get('NumberOfTemporalPositions', 'N/A')}")


{'File': '1.2.840.113619.2.514.11554579.1340171.29576.1733166234.112.MR.dcm', 'InstanceNumber': '1', 'PhaseEncodingDirection': 'ROW', 'TotalReadoutTime': 'N/A'}
{'File': '1.2.840.113619.2.514.11554579.1340171.29576.1733166234.113.MR.dcm', 'InstanceNumber': '2', 'PhaseEncodingDirection': 'ROW', 'TotalReadoutTime': 'N/A'}
{'File': '1.2.840.113619.2.514.11554579.1340171.29576.1733166234.114.MR.dcm', 'InstanceNumber': '3', 'PhaseEncodingDirection': 'ROW', 'TotalReadoutTime': 'N/A'}
{'File': '1.2.840.113619.2.514.11554579.1340171.29576.1733166234.115.MR.dcm', 'InstanceNumber': '4', 'PhaseEncodingDirection': 'ROW', 'TotalReadoutTime': 'N/A'}
{'File': '1.2.840.113619.2.514.11554579.1340171.29576.1733166234.116.MR.dcm', 'InstanceNumber': '5', 'PhaseEncodingDirection': 'ROW', 'TotalReadoutTime': 'N/A'}
{'File': '1.2.840.113619.2.514.11554579.1340171.29576.1733166234.117.MR.dcm', 'InstanceNumber': '6', 'PhaseEncodingDirection': 'ROW', 'TotalReadoutTime': 'N/A'}
{'File': '1.2.840.113619.2.514.115